# Quantum Process Tomography via Genetic Algorithms

## Single SU(2) Transformations
___

This notebook performs the process tomography for the numerical experiments proposed in our paper 'Retrieving complex polarization transformations via optimized quantum process tomography'.
 

   The notebook is organized as follows: 

   1. Importing the synthetic data
   2. Setting the hyper-parameters of the Genetic Algorithm
   3. Performing the genetic recostruction with 6 measurements
   4. Performing the genetic recostruction with 5 measurements
   5. Performing the genetic recostruction with 4 measurements
   6. Performing the genetic recostruction with 3 measurements


________

First, the required libraries are imported:

In [ ]:
from deap import base
from deap import creator
from deap import tools
import numpy as np
import random
import math
import pandas as pd
import time
import matplotlib.pyplot as plt
import statistics
import utils.GA_utils as GA_utils

By running the following cell, you import the experimental data.


In [ ]:
# Insert Correct Path
path = "dataset_gaussian"
LL = np.loadtxt(path + "/LL.txt", dtype="f", delimiter="\t")
HH = np.loadtxt(path + "/HH.txt", dtype="f", delimiter="\t")
HL = np.loadtxt(path + "/HL.txt", dtype="f", delimiter="\t")
LD = np.loadtxt(path + "/LD.txt", dtype="f", delimiter="\t")
LH = np.loadtxt(path + "/LH.txt", dtype="f", delimiter="\t")
HD = np.loadtxt(path + "/HD.txt", dtype="f", delimiter="\t")
DL = np.loadtxt(path + "/DL.txt", dtype="f", delimiter="\t")

If available, you can import the theoretical $U_{Th}$ to compute the fidelity of the reconstructed process.

In [ ]:
Theta_t = np.loadtxt(path + "/random_Theta.txt", dtype="f", delimiter="\t")
nx_t = np.loadtxt(path + "/random_nx.txt", dtype="f", delimiter="\t")
ny_t = np.loadtxt(path + "/random_ny.txt", dtype="f", delimiter="\t")
nz_t = np.loadtxt(path + "/random_nz.txt", dtype="f", delimiter="\t")

The function `compute_unitary` is used to compute the unitary $U$, given the parameters $\Theta\in[0,\pi]$ and $\mathbf{n}=(n_x,n_y,n_z)$ according to:

\begin{equation}
U=\begin{pmatrix}
\cos \Theta -i \sin \Theta \,n_z && -i\sin \Theta \,(n_x-i n_y)\\
-i\sin \Theta \,(n_x+i n_y) && \cos \Theta + i \sin \Theta \,n_z
\end{pmatrix}
\end{equation}

In [ ]:
def compute_unitary(Theta, nx, ny, nz):
    I = np.array([[1, 0], [0, 1]])
    sx = np.matrix([[0, 1], [1, 0]])
    sy = np.matrix([[0, -1j], [1j, 0]])
    sz = np.matrix([[1, 0], [0, -1]])
    return math.cos(Theta) * I - 1j * math.sin(Theta) * (nx * sx + ny * sy + nz * sz)

The hyper-parameters of the GA are defined:

In [ ]:
POP_SIZE = 60
CXPB = 0.8
MUTPB = 0.3
NGEN = 100
STATS = GA_utils.createStats()
pop_list = None
TS_SIZE = 3

_____

### Genetic Reconstruction with 6 measurements

The following cell defines the fitness function to perform the genetic reconstruction with 6 measurements. 
The set of measurements is  $[HH, LL, HL, LD, LH, HD]$

In [ ]:
def evaluate(HH, LL, HL, LD, LH, HD, individual):
    '''
    Fitness function for genetic reconstruction with 6 measurements
    _____
    
    :return The fitness value of an individual
    '''
    Theta, nx, ny, nz = individual[0], individual[1], individual[2], individual[3]
    f = ((math.cos(Theta)**2 - HH + nx**2 * math.sin(Theta)**2)**2) / HH + ((math.cos(Theta)**2 - LL + nz**2 * math.sin(Theta)**2)**2) / LL + ( ( -HL + 1/2 * (math.cos(Theta)**2 - 2 * ny * math.cos(Theta) * math.sin(Theta) + (ny**2 + (nx + nz)**2) * math.sin(Theta)**2))**2 )/ HL + ( ( -LD + 1/2 * (math.cos(Theta)**2 - 2 * nx * math.cos(Theta) * math.sin(Theta) + (nx**2 + (ny + nz)**2) * math.sin(Theta)**2))**2 ) / LD + ((-LH + 1/2 * (math.cos(Theta)**2 + (ny**2 + (nx + nz)**2) * math.sin(Theta)**2 + ny * math.sin(2 * Theta)))**2 )/ LH + (( -HD + 1/2 * (math.cos(Theta)**2 + ((nx + ny)**2 + nz**2) * math.sin(Theta)**2 +nz * math.sin(2 * Theta)))**2)/ HD
    return f

The reconstruction starts. Specify in  `iteration` the number of independent runs to compute. Please, specify also the  `number_of_processes` you have to reconstruct, depending on the size of your dataset. 

In [ ]:
# Create the DEAP toolbox
toolbox = GA_utils.createToolbox()
toolbox.decorate("mutate", GA_utils.checkBounds(0, np.pi))
toolbox.decorate("mate", GA_utils.checkBounds(0, np.pi))

# Set the number of iterations used for averaging the GA results
iterations = 10

#Specify the number of process you have to reconstruct
number_of_processes = 1000

# Reconstruction
F_to_average = []
for _ in range(iterations):
    best, times = [], []
    fidelities = []
    for i in range(number_of_processes):

        toolbox.register("evaluate", evaluate, HH[i], LL[i], HL[i], LD[i], LH[i], HD[i])

        GA = GA_utils.updatedGA(
            toolbox,
            pop_size=POP_SIZE,
            cxpb=CXPB,
            mutpb=MUTPB,
            ngen=NGEN,
            stats=STATS,
            tourn_size=TS_SIZE,
            hof=tools.HallOfFame(1),
            verbose=False,
        )
        best.append(GA[2][0])

        individual = GA[2][0]
        Theta = individual[0]
        nx = individual[1]
        ny = individual[2]
        nz = individual[3]

        computed_u = compute_unitary(Theta, nx, ny, nz)
        data_u = compute_unitary(Theta_t[i], nx_t[i], ny_t[i], nz_t[i])

        F = 0.5 * np.linalg.norm(np.trace(data_u.getH() * computed_u))
        fidelities.append(F)
    F_to_average.append(fidelities)

# Averaging of results
avg = []
for process in range(number_of_processes):
    fid = []
    for _ in range(iterations):
        fid.append(F_to_average[_][process])
    avg.append(statistics.mean(fid))


The fidelities of individual reconstructions is shown:

In [ ]:
plt.plot(range(number_of_processes), avg)
plt.show()

The mean infidelity is computed:

In [ ]:
print(
    statistics.mean([1 - avg[i] for i in range(len(avg))]),
    statistics.stdev([1 - avg[i] for i in range(len(avg))]),
)

____

### Genetic Reconstruction with 5 measurements

The following cell defines the fitness function to perform the genetic reconstruction with 5 measurements. 
The set of measurements is $[HL, LD, LL, DL, LH]$.

In [ ]:
def evaluate(HL, LD, LL, DL, LH, individual):
    '''
    Fitness function for genetic reconstruction with 5 measurements
    _____
    
    :return The fitness value of an individual
    '''
    Theta, nx, ny, nz = individual[0], individual[1], individual[2], individual[3]
    f = (0.5 - HL - ny * math.cos(Theta) * math.sin(Theta) + nx * nz * math.sin(Theta)**2)**2 / HL + (
                0.5 - LD - nx * math.cos(Theta) * math.sin(Theta) + ny * nz * math.sin(Theta)**2)**2 / LD + (
                    math.cos(Theta)**2 - LL + nz**2 * math.sin(Theta)**2)**2 / LL + (
                    -DL + 0.5* (1 + 2 * ny * nz * math.sin(Theta)**2 + nx * math.sin(2 * Theta)))**2 / DL + (
                        -LH + 1 / 2 * (1 + 2 * nx * nz * math.sin(Theta)**2 + ny * math.sin(2 * Theta)))**2 / LH
    return f

In [ ]:
# Create the DEAP toolbox
toolbox = GA_utils.createToolbox()
toolbox.decorate("mutate", GA_utils.checkBounds(0, np.pi))
toolbox.decorate("mate", GA_utils.checkBounds(0, np.pi))

# Set the number of iterations used for averaging the GA results
iterations = 10

#Specify the number of process you have to reconstruct
number_of_processes = 1000

# Reconstruction
F_to_average = []
for _ in range(iterations):

    best = []

    for i in range(number_of_processes):
        toolbox.register("evaluate", evaluate, HL[i], LD[i], LL[i], DL[i], LH[i])

        GA = GA_utils.updatedGA(
            toolbox,
            pop_size=POP_SIZE,
            cxpb=CXPB,
            mutpb=MUTPB,
            ngen=NGEN,
            stats=STATS,
            tourn_size=TS_SIZE,
            hof=tools.HallOfFame(1),
            verbose=True,
        )
        best.append(GA[2][0])
    fidelities = []

    for i in range(len(best)):
        individual = best[i]
        Theta = individual[0]
        nx = individual[1]
        ny = individual[2]
        nz = individual[3]

        computed_u = compute_unitary(Theta, nx, ny, nz)
        data_u = compute_unitary(Theta_t[i], nx_t[i], ny_t[i], nz_t[i])

        F = 0.5 * np.linalg.norm(np.trace(data_u.getH() * computed_u))
        fidelities.append(F)
    F_to_average.append(fidelities)

#Averaging for the results
avg = []
for process in range(number_of_processes):
    fid = []
    for _ in range(iterations):
        fid.append(F_to_average[_][process])
    avg.append(statistics.mean(fid))



Individual fidelities are shown and the mean infidelity is computed:

In [ ]:
plt.plot(range(number_of_processes), avg)
plt.show()
print(statistics.mean([1 - avg[i] for i in range(len(avg))]), statistics.stdev(avg))

____

### Genetic Reconstruction with 4 measurements

The following cell defines the fitness function to perform the genetic reconstruction with 4 measurements. 
The set of measurements is $[HH, LL, LH, HD]$.

In [ ]:
def evaluate(HH, LL, LH, HD,  individual):
    '''
    Fitness function for genetic reconstruction with 4 measurements
    _____
    
    :return The fitness value of an individual
    '''
    Theta, nx, ny, nz = individual[0], individual[1], individual[2], individual[3]
    f = ((math.cos(Theta)**2 - HH + nx**2 * math.sin(Theta)**2)**2) / HH + ((math.cos(Theta)**2 - LL + nz**2 * math.sin(Theta)**2)**2) / LL  + ((-LH + 1/2 * (math.cos(Theta)**2 + (ny**2 + (nx + nz)**2) * math.sin(Theta)**2 + ny * math.sin(2 * Theta)))**2 )/ LH + (( -HD + 1/2 * (math.cos(Theta)**2 + ((nx + ny)**2 + nz**2) * math.sin(Theta)**2 +nz * math.sin(2 * Theta)))**2)/ HD
    return f

In [ ]:
# Create the DEAP toolbox
toolbox = GA_utils.createToolbox()
toolbox.decorate("mutate", GA_utils.checkBounds(0, np.pi))
toolbox.decorate("mate", GA_utils.checkBounds(0, np.pi))

# Set the number of iterations used for averaging the GA results
iterations = 10

#Specify the number of process you have to reconstruct
number_of_processes = 1000

# Reconstruction
F_to_average = []
for _ in range(iterations):

    best = []
    fidelities = []
    for i in range(number_of_processes):
        toolbox.register("evaluate", evaluate, HH[i], LL[i], LH[i], HD[i])

        GA = GA_utils.updatedGA(
            toolbox,
            pop_size=POP_SIZE,
            cxpb=CXPB,
            mutpb=MUTPB,
            ngen=NGEN,
            stats=STATS,
            tourn_size=TS_SIZE,
            hof=tools.HallOfFame(1),
            verbose=False,
        )
        best.append(GA[2][0])
        individual = GA[2][0]
        Theta = individual[0]
        nx = individual[1]
        ny = individual[2]
        nz = individual[3]
        computed_u = compute_unitary(Theta, nx, ny, nz)
        data_u = compute_unitary(Theta_t[i], nx_t[i], ny_t[i], nz_t[i])

        F = 0.5 * np.linalg.norm(np.trace(data_u.getH() * computed_u))
        fidelities.append(F)
    F_to_average.append(fidelities)
    
#Averaging The Results
avg = []
for process in range(number_of_processes):
    fid = []
    for _ in range(iterations):
        fid.append(F_to_average[_][process])
    avg.append(statistics.mean(fid))

Individual fidelities are shown and the mean infidelity is computed:

In [ ]:
plt.plot(range(number_of_processes), avg)
plt.show()
print(statistics.mean([1 - avg[i] for i in range(len(avg))]), statistics.stdev(avg))

____

### Genetic Reconstruction with 3 measurements

The following cell defines the fitness function to perform the genetic reconstruction with 3 measurements. 
The set of measurements is $[LH, HD, DL]$.

In [ ]:
def evaluate(LH, HD, DL, individual):
    '''
    Fitness function for genetic reconstruction with 3 measurements
    _____
    
    :return The fitness value of an individual
    '''
    Theta, nx, ny, nz = individual[0], individual[1], individual[2], individual[3]
    f = ((-LH + 1/2 * (math.cos(Theta)**2 + (ny**2 + (nx + nz)**2) * math.sin(Theta)**2 + ny * math.sin(2 * Theta)))**2 )/ LH + (( -HD + 1/2 * (math.cos(Theta)**2 + ((nx + ny)**2 + nz**2) * math.sin(Theta)**2 +nz * math.sin(2 * Theta)))**2)/ HD + ( -DL + 1/2 * (math.cos(Theta)**2 + (nx**2 + (ny + nz)**2) * math.sin(Theta)**2 + nx * math.sin(2 * Theta)))**2/DL    
    return f

In [ ]:
# Create the DEAP toolbox
toolbox = GA_utils.createToolbox()
toolbox.decorate("mutate", GA_utils.checkBounds(0, np.pi))
toolbox.decorate("mate", GA_utils.checkBounds(0, np.pi))

# Set the number of iterations used for averaging the GA results
iterations = 10

#Specify the number of process you have to reconstruct
number_of_processes = 1000

# Reconstruction
F_to_average = []
for _ in range(iterations):
    fidelities = []
    best = []

    for i in range(number_of_processes):
        toolbox.register("evaluate", evaluate, LH[i], HD[i], DL[i])

        GA = GA_utils.updatedGA(
            toolbox,
            pop_size=POP_SIZE,
            cxpb=CXPB,
            mutpb=MUTPB,
            ngen=NGEN,
            stats=STATS,
            tourn_size=TS_SIZE,
            hof=tools.HallOfFame(1),
            verbose=False,
        )
        best.append(GA[2][0])

        individual = GA[2][0]
        Theta = individual[0]
        nx = individual[1]
        ny = individual[2]
        nz = individual[3]
        computed_u = compute_unitary(Theta, nx, ny, nz)
        data_u = compute_unitary(Theta_t[i], nx_t[i], ny_t[i], nz_t[i])

        F = 0.5 * np.linalg.norm(np.trace(data_u.getH() * computed_u))
        fidelities.append(F)
    F_to_average.append(fidelities)


# Averaging of results
avg = []
for process in range(number_of_processes):
    fid = []
    for _ in range(iterations):
        fid.append(F_to_average[_][process])
    avg.append(statistics.mean(fid))

Individual fidelities are shown and the mean infidelity is computed:

In [ ]:
plt.plot(range(number_of_processes), avg)
plt.show()
print(statistics.mean([1 - avg[i] for i in range(len(avg))]), statistics.stdev(avg))